# Tutorial CS006: Probabilistic Load Forecasting

This tutorial demonstrates how to implement probabilistic energy forecasting using Quantile Regression. You will learn how to:
1. Estimate forecast uncertainty using multiple quantiles.
2. Implement the Pinball (Quantile) Loss function.
3. Visualize confidence intervals and uncertainty bands.

In [ ]:
import sys
import os
import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

sys.path.append(os.path.abspath('../../src'))
from models.probabilistic import ProbabilisticLSTM, QuantileLoss
from preprocessing.pipeline import DataPipeline
from utils.metrics import continuous_ranked_probability_score
from utils.plotting import plot_probabilistic_forecast, set_style
from utils.trainer import Trainer
from data.download import download_public_dataset

set_style()

In [ ]:
data_path = download_public_dataset('D001', save_dir='../../data/raw')
df = pd.read_csv(data_path)
pipeline = DataPipeline()
train_df, val_df, test_df = pipeline.split_data(df)

target_col = 'load'
train_scaled = pipeline.fit_transform(train_df, [target_col])
val_scaled = pipeline.transform(val_df, [target_col])
test_scaled = pipeline.transform(test_df, [target_col])

WINDOW_SIZE = 24
X_train, y_train = pipeline.create_sequences(train_scaled, WINDOW_SIZE)
X_val, y_val = pipeline.create_sequences(val_scaled, WINDOW_SIZE)
X_test, y_test = pipeline.create_sequences(test_scaled, WINDOW_SIZE)

train_loader = DataLoader(TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float()), batch_size=64, shuffle=True)
val_loader = DataLoader(TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).float()), batch_size=64)
test_loader = DataLoader(TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float()), batch_size=64)

In [ ]:
QUANTILE_LEVELS = [0.1, 0.5, 0.9]
model = ProbabilisticLSTM(input_size=1, hidden_size=64, num_layers=2, num_quantiles=len(QUANTILE_LEVELS))
criterion = QuantileLoss(QUANTILE_LEVELS)
trainer = Trainer(model, criterion)
trainer.train(train_loader, val_loader, epochs=5, save_path='prob_best.pth')

In [ ]:
y_pred_scaled = trainer.predict(test_loader)
y_true = pipeline.scaler.inverse_transform(y_test)

# Inverse transform each quantile individually
y_pred_quantiles = np.zeros_like(y_pred_scaled)
for i in range(len(QUANTILE_LEVELS)):
    y_pred_quantiles[:, i] = pipeline.scaler.inverse_transform(y_pred_scaled[:, i].reshape(-1, 1)).flatten()

crps = continuous_ranked_probability_score(y_true, y_pred_quantiles, QUANTILE_LEVELS)
print(f"Test CRPS: {crps:.4f}")

plot_probabilistic_forecast(y_true[:168], y_pred_quantiles[:168], QUANTILE_LEVELS, 
                             title="Probabilistic Load Forecast (10-90% CI)", save_path='../../figures/cs006_prob_forecast.pdf')